In [1]:
from custom.datasets import Meiosis_Dataset
from custom.models import Net
from custom.utils import accuracy, weight_balance
import torch
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import ConcatDataset, DataLoader
from torch import optim
import numpy as np

In [2]:
# dataset = Meiosis_Dataset(path='../SC_figures', label={'leptotene', 'zygotene', 'pachytene_ab', 'pachytene_n'}, target=[0,0,0,0])

In [3]:
dataset_1 = Meiosis_Dataset(path='../SC_figures', label='zygotene', target=0)
indicies_1, indicies_2 = random_sample(len(dataset_1), .1)
dataset_1_t = Meiosis_Dataset(path='../SC_figures', label='zygotene', target=0)
dataset_1_t.im_paths = [dataset_1_t.im_paths[index] for index in indicies_1]
dataset_1_v = Meiosis_Dataset(path='../SC_figures', label='zygotene', target=0)
dataset_1_v.im_paths = [dataset_1_v.im_paths[index] for index in indicies_2]

dataset_2 = Meiosis_Dataset(path='../SC_figures', label='pachytene_ab', target=1)
indicies_1, indicies_2 = random_sample(len(dataset_2), .1)
dataset_2_t = Meiosis_Dataset(path='../SC_figures', label='pachytene_ab', target=1)
dataset_2_t.im_paths = [dataset_2_t.im_paths[index] for index in indicies_1]
dataset_2_v = Meiosis_Dataset(path='../SC_figures', label='pachytene_ab', target=1)
dataset_2_v.im_paths = [dataset_2_v.im_paths[index] for index in indicies_2]

dataset_t = ConcatDataset([dataset_1_t, dataset_2_t])
train_loader = DataLoader(dataset_t, batch_size=64, shuffle=True)

dataset_v = ConcatDataset([dataset_1_v, dataset_2_v])
val_loader = DataLoader(dataset_v, batch_size=8)

In [4]:
print(len(dataset_1), len(dataset_2), len(dataset))
weights = weight_balance([len(dataset_1), 
                          len(dataset_2)])
weights = torch.FloatTensor(weights)
print(weights)

129 209 338

 1.3101
 0.8086
[torch.FloatTensor of size 2]



In [5]:
model = Net(2)
model.cuda()

Net(
  (conv1a): conv_bn(
    (conv): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), dilation=(2, 2))
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
  )
  (conv1b): conv_bn(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), dilation=(2, 2))
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
  )
  (conv2): inception_v2(
    (conv3a): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
    (conv3b): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv5a): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
    (conv5b): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv5c): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv7a): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
    (conv7b): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv7c): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv7d

In [6]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
# optimizer = optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)

In [7]:
loss_func = F.cross_entropy

In [8]:
def train(epoch):
    model.train()
    train_loss = 0
    acc = 0
    for batch_idx, (im, target) in enumerate(train_loader):
        input, target, weight = Variable(im).cuda(), Variable(target.squeeze_()).cuda(), Variable(weights).cuda()
        optimizer.zero_grad()
        output = model(input)
        loss = loss_func(output, target, weight)
        loss.backward()
        acc += accuracy(output, target).data[0]
        train_loss += loss.data[0]
        optimizer.step()
        
    print('====> Epoch: {} Average loss: {:.4f} Average acc: {:.3f}'.format(
          epoch, train_loss / len(train_loader), acc / len(train_loader)))

In [9]:
from torchvision.utils import save_image
from torch.nn.functional import threshold
def test(epoch):
    model.eval()
    test_loss = 0
    for i, (im, target) in enumerate(val_loader):
        input, target, weight = Variable(im).cuda(), Variable(target.squeeze_()).cuda(), Variable(weights).cuda()
        output = model(input)
        loss += loss_func(output, target, weight)
        acc += accuracy(output, target).data[0]
        test_loss += loss.data[0]
        
    print('====> Epoch: {} Average loss: {:.4f} Average acc: {:.3f}'.format(
          epoch, test_loss / len(val_loader), acc / len(val_loader)))

In [10]:
torch.cuda.empty_cache()
epochs = 30
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)
#


/usr/local/anaconda3/lib/python3.6/site-packages/skimage/morphology/misc.py:122: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  warn("Only one label was provided to `remove_small_objects`. "


====> Epoch: 1 Average loss: 0.6997 Average acc: 0.558
====> Epoch: 2 Average loss: 0.6798 Average acc: 0.604
====> Epoch: 3 Average loss: 0.6645 Average acc: 0.630
====> Epoch: 4 Average loss: 0.6395 Average acc: 0.637
====> Epoch: 5 Average loss: 0.6424 Average acc: 0.662
====> Epoch: 6 Average loss: 0.6289 Average acc: 0.623
====> Epoch: 7 Average loss: 0.6045 Average acc: 0.690
====> Epoch: 8 Average loss: 0.5796 Average acc: 0.700
====> Epoch: 9 Average loss: 0.6069 Average acc: 0.682
====> Epoch: 10 Average loss: 0.5662 Average acc: 0.766
====> Epoch: 11 Average loss: 0.5815 Average acc: 0.706
====> Epoch: 12 Average loss: 0.5578 Average acc: 0.718
====> Epoch: 13 Average loss: 0.5539 Average acc: 0.697
====> Epoch: 14 Average loss: 0.4924 Average acc: 0.797
====> Epoch: 15 Average loss: 0.5355 Average acc: 0.767
====> Epoch: 16 Average loss: 0.4866 Average acc: 0.768
====> Epoch: 17 Average loss: 0.4721 Average acc: 0.782
====> Epoch: 18 Average loss: 0.5263 Average acc: 0.768
=

In [11]:
torch.save(model.state_dict(), './models/period_classification_1')